In [1]:
from PIL import Image
import numpy as np
from func import *
import os
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

c:\Users\Igorexy\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Откройте изображение с помощью Pillow
image = Image.open('dataset/Идет.jpeg')

# Преобразуйте изображение в черно-белое
bw_image = image.convert('L')

# Преобразуйте изображение в матрицу numpy
bw_array = np.array(bw_image)

# Создайте матрицу из нулей и единиц
binary_matrix = (bw_array < 128).astype(int)

# Теперь binary_matrix содержит вашу матрицу из нулей и единиц,
# где единицы соответствуют черным пикселям

# Вы можете сохранить эту матрицу в файл или продолжить работу с ней в коде


In [3]:
# # Откройте изображение
# image = Image.open('dataset/Смотрит влево 01.jpg')

# # Преобразуйте изображение в черно-белое
# bw_image = image.convert(mode='1')

# # Преобразуйте изображение в массив numpy
# bw_array = np.array(image, dtype=np.int16)/255

# # Матрица bw_array будет содержать 0 (белый пиксель) и 1 (черный пиксель)
# print(bw_array)


In [4]:
# # Преобразуйте матрицу в изображение
# bw_image = Image.fromarray(abs(binary_matrix-1).astype('uint8') * 255)

# # Сохраните изображение в формате JPG
# bw_image.save('новое_изображение.jpg')

In [5]:
path = 'dataset'
files = os.listdir(path)

output = []
for file in files:
    image = Image.open('dataset/' + file)

    # Преобразуйте изображение в черно-белое
    bw_image = image.convert('L')

    # Преобразуйте изображение в матрицу numpy
    bw_array = np.array(bw_image)

    # Создайте матрицу из нулей и единиц
    binary_matrix = (bw_array < 128).astype(int)
    # output.append(abs(binary_matrix-1))
    output.append(binary_matrix)

files = [file.split('.')[0] for file in files]

In [6]:
stop_words = ['01']
for i in range(len(files)):
    for s_w in stop_words:
        files[i] = files[i].replace(s_w, '')
img_name = lemmatize(files)

vect = TfidfVectorizer(use_idf=True)
tf_idf = vect.fit_transform(img_name).toarray()

In [7]:
words = vect.get_feature_names_out()
print(words)
print(tf_idf)

['влево' 'вправо' 'идти' 'красться' 'приземляться' 'смотреть']
[[0.         0.         1.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.        ]
 [0.82219037 0.         0.         0.         0.         0.56921261]
 [0.         0.7640961  0.         0.         0.         0.64510243]
 [0.         0.7640961  0.         0.         0.         0.64510243]]


In [8]:
print(len(output), output[0].shape)
y_output = np.array([img.flatten() for img in output])

6 (125, 125)


In [9]:
[print(len(output[i])) for i in range(len(output))]

125
125
125
125
125
125


[None, None, None, None, None, None]

In [10]:
# Установите желаемое количество ядер процессора
num_cores = 2  # Замените этот числовой параметр на количество ядер, которое вы хотите использовать

# Создайте объект сессии TensorFlow
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                                  inter_op_parallelism_threads=num_cores,
                                  allow_soft_placement=True,
                                  device_count={'CPU': num_cores})

# Примените конфигурацию к сессии TensorFlow
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

# Теперь ваша нейронная сеть Keras будет использовать указанное количество ядер процессора

In [11]:
input_dim = len(vect.get_feature_names_out())

# Создаем модель
model = Sequential()

# Добавляем слои
model.add(Dense(units=10, input_dim=input_dim, activation='relu'))
# model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=32, activation='relu'))
# model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=128, activation='relu'))
# model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=4096, activation='relu'))
model.add(Dense(units=8192, activation='relu'))
# model.add(Dense(units=65536, activation='relu'))
model.add(Dense(units=len(y_output[0]), activation='linear')) 

# Компилируем модель
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error', 'mean_absolute_error'])

In [12]:
model.fit(tf_idf, y_output, epochs=70, batch_size=32, validation_split=0.2)

Epoch 1/70
1/1 [==============================] - 5s 5s/step - loss: 0.2278 - mean_squared_error: 0.2278 - mean_absolute_error: 0.2295 - val_loss: 0.2648 - val_mean_squared_error: 0.2648 - val_mean_absolute_error: 0.2670
Epoch 2/70
1/1 [==============================] - 2s 2s/step - loss: 0.2260 - mean_squared_error: 0.2260 - mean_absolute_error: 0.2292 - val_loss: 0.2618 - val_mean_squared_error: 0.2618 - val_mean_absolute_error: 0.2682
Epoch 3/70
1/1 [==============================] - 3s 3s/step - loss: 0.2214 - mean_squared_error: 0.2214 - mean_absolute_error: 0.2302 - val_loss: 0.2502 - val_mean_squared_error: 0.2502 - val_mean_absolute_error: 0.2716
Epoch 4/70
1/1 [==============================] - 2s 2s/step - loss: 0.2039 - mean_squared_error: 0.2039 - mean_absolute_error: 0.2332 - val_loss: 0.2114 - val_mean_squared_error: 0.2114 - val_mean_absolute_error: 0.2821
Epoch 5/70
1/1 [==============================] - 2s 2s/step - loss: 0.1488 - mean_squared_error: 0.1488 - mean_abso

In [15]:
# model.save("first_model_data6.h5")

In [13]:
text_prompt = 'крадется'

In [14]:
# stop_words = ['01']
# for i in range(len(files)):
#     for s_w in stop_words:
#         files[i] = files[i].replace(s_w, '')

prompt = lemmatize([[text_prompt]])

# vect = TfidfVectorizer(use_idf=True)
tf_idf_prompt = vect.transform(prompt).toarray()

words = vect.get_feature_names_out()
print(words)
print(tf_idf_prompt)

generation = model.predict(tf_idf_prompt)
generation = generation.reshape(125, 125)

for i in range(len(generation)):
    for j in range(len(generation[i])):
        generation[i][j] = round(generation[i][j])

inversing_gen = 1 - generation       

# Преобразование бинарной матрицы обратно в изображение
bw_image = Image.fromarray((inversing_gen * 255).astype(np.uint8), 'L')
# Сохранение изображения в файл
bw_image.save(f'output/{text_prompt}.jpg')

['влево' 'вправо' 'идти' 'красться' 'приземляться' 'смотреть']
[[0. 0. 0. 1. 0. 0.]]
1/1 [==============================] - 0s 226ms/step
